In [1]:
import os
os.chdir("../")
%pwd

'e:\\Deep Learning\\TENSORFLOW\\rice_image_detection'

In [2]:
from dataclasses import dataclass
from pathlib import Path
@dataclass(frozen= True)
class ModelTrainingConfig:
    root_dir: Path
    model_dir: Path
    trained_model_dir: Path
    history_dir: Path
    epochs: int


In [3]:
from src.RICE_IMAGE_DETECTION.constants import *
from src.RICE_IMAGE_DETECTION.utils.common import read_yaml, create_directories

In [4]:
class ConfigureationManager:
    def __init__(self,
            config_filepath = CONFIG_FILE_PATH,
            params_filepath = PARAMS_FILE_PATH):
        
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])

    def get_model_training_config(self) -> ModelTrainingConfig:
        config = self.config.training

        create_directories([config.root_dir])

        model_training_config = ModelTrainingConfig(
            root_dir = Path(config.root_dir),
            model_dir = Path(config.model_dir),
            trained_model_dir = Path(config.trained_model_dir),
            history_dir= Path(config.history_dir),
            epochs = self.params.epochs,
        )

        return model_training_config

In [5]:
import os
import json
import tensorflow as tf
from tensorflow import keras
from src.RICE_IMAGE_DETECTION import logger

In [6]:
from src.RICE_IMAGE_DETECTION.components.data_preparation import DataPreparation
from src.RICE_IMAGE_DETECTION.config.configuration import ConfigureationManager

In [7]:
class ModelTraining:
    def __init__(self, config: ModelTrainingConfig):
        self.config = config

    def get_model(self):
        logger.info("-----------Model loaded from the model path----------")
        model_dir = self.config.model_dir
        cnn = tf.keras.models.load_model(model_dir)
        return cnn
    
    def train(self, model, training_set, test_set):
        logger.info("-----------Model training is beginning----------")
        history = model.fit(training_set, epochs=1, validation_data=test_set)
        logger.info("-----------Model training is ending----------")
        return model, history
    
    def save_model(self, model):
        trained_model_dir = self.config.trained_model_dir
        model.save(trained_model_dir)
        logger.info("--------Model saved successfully--------")

    def save_history(self, history):
        history_dir = self.config.history_dir
        history_dict = history.history
        
        with open(history_dir, 'w') as f:
            json.dump(history_dict, f)
        
        logger.info(f"------History saved successfully at {history_dir}---------")
    def all_modules(self, model, history):
        return model, history

In [8]:
try:
    config = ConfigureationManager()
    #data preparation
    data_preparation_config = config.get_data_preparation_config()
    data_preparation = DataPreparation(config=data_preparation_config)
    training_set, test_set = data_preparation.train_test_set()
    #model_training
    model_training_config = config.get_model_training_config()
    model_training = ModelTraining(config=model_training_config)
    cnn = model_training.get_model()
    trained_model, history = model_training.train(model=cnn, training_set=training_set, test_set=test_set)
    model_training.save_model(model=trained_model)
    model_training.save_history(history=history)
    trained_model, history = model_training.all_modules(trained_model, history)
except Exception as e:
    raise e

[2024-07-01 15:35:53,562: INFO: common: yaml file: config\config.yaml loaded successfully]
[2024-07-01 15:35:53,562: INFO: common: yaml file: params.yaml loaded successfully]
[2024-07-01 15:35:53,569: INFO: common: created directory at: artifacts]
[2024-07-01 15:35:53,570: INFO: common: created directory at: artifacts/data_preparation]
[2024-07-01 15:35:53,571: INFO: data_preparation: -------The train and test datagen created-------]


Found 60000 images belonging to 5 classes.
Found 15000 images belonging to 5 classes.
[2024-07-01 15:35:58,822: INFO: data_preparation: -------The test and train set created-------]
[2024-07-01 15:35:58,823: INFO: common: created directory at: artifacts/training]
[2024-07-01 15:35:58,823: INFO: 3736226327: -----------Model loaded from the model path----------]
[2024-07-01 15:35:59,222: INFO: 3736226327: -----------Model training is beginning----------]
1875/1875 [==============================] - 1373s 732ms/step - loss: 0.1649 - accuracy: 0.9587 - val_loss: 0.2316 - val_accuracy: 0.9259
[2024-07-01 15:58:53,333: INFO: 3736226327: -----------Model training is ending----------]
[2024-07-01 15:58:53,539: INFO: 3736226327: --------Model saved successfully--------]
[2024-07-01 15:58:53,543: INFO: 3736226327: ------History saved successfully at artifacts\training\history.json---------]
